In [1]:
import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import fiona
from pyproj import Proj, transform
import osmnx as ox
import networkx as nx
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
nx.__version__

In [ ]:
conn = psycopg2.connect(dbname="gis", user="postgres", password="")
ox.config(log_file=True, log_console=True, use_cache=True)

## Graph from polygon - Rapperswil

In [ ]:
rapperswil_polygon_query = "SELECT way AS geometry FROM planet_osm_polygon WHERE osm_id = -1683921"

polygon_gdf = gpd.read_postgis(rapperswil_polygon_query, conn, geom_col='geometry')
polygon_gdf.crs = fiona.crs.from_epsg('3857')
polygon_gdf = polygon_gdf.to_crs(fiona.crs.from_epsg('4326'))
polygon = polygon_gdf['geometry'][0]

In [ ]:
rapperswil_graph = ox.graph_from_polygon(polygon, network_type='all')
rapperswil_graph_projected = ox.project_graph(rapperswil_graph)
fig, ax = ox.plot_graph(rapperswil_graph_projected)

In [ ]:
ox.plot_graph_folium(rapperswil_graph, popup_attribute='name', edge_width=2)

## Graph from bounding box - Rapperswil

In [ ]:
# define a bounding box of Rapperswil Altstadt
west, south, east, north = 8.8130341262,47.2231736868,8.826466629,47.2278149394
# west, south, east, north = 8.525952,47.3705,8.545264,47.383228 # Zürich small
west, south, east, north = 8.448006,47.320221,8.625442,47.434666 # Zürich big
G = ox.graph_from_bbox(north, south, east, west, network_type='all')
ox.plot_graph_folium(G, popup_attribute='name', edge_width=2)

In [ ]:
node_centrality = nx.closeness_centrality(G)

In [ ]:
# plot it
df = pd.DataFrame(data=pd.Series(node_centrality).sort_values(), columns=['cc'])
df['colors'] = ox.get_colors(n=len(df), cmap='inferno', start=0.2)
df = df.reindex(G.nodes())
nc = df['colors'].tolist()
fig, ax = ox.plot_graph(G, bgcolor='k', node_size=30, node_color=nc, node_edgecolor='none', node_zorder=2,
                        edge_color='#555555', edge_linewidth=1.5, edge_alpha=1)

In [ ]:
#G = ox.project_graph(G)

In [ ]:
# edge closeness centrality: convert graph to a line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(G))

# list of edge values for the orginal graph
ev = [edge_centrality[edge + (0,)] for edge in G.edges()]

# color scale converted to list of colors for graph edges
norm = colors.Normalize(vmin=min(ev)*0.8, vmax=max(ev))
cmap = cm.ScalarMappable(norm=norm, cmap=cm.inferno)
ec = [cmap.to_rgba(cl) for cl in ev]

# color the edges in the original graph with closeness centralities in the line graph
fig, ax = ox.plot_graph(G, bgcolor='k', axis_off=True, node_size=0, node_color='w', node_edgecolor='gray', node_zorder=2,
                        edge_color=ec, edge_linewidth=1.5, edge_alpha=1)

## Graph from bounding box - Zürich

In [ ]:
# define a bounding box of Zürich
west, south, east, north = 8.531445,47.370005,8.546552,47.379712
G1 = ox.graph_from_bbox(north, south, east, west, network_type='all')
ox.plot_graph_folium(G1, popup_attribute='name', edge_width=2)

In [5]:
G = ox.graph_from_file('/data/switzerland-latest.osm.bz2', network_type='drive')

KeyboardInterrupt: 